In [2]:
#import the necessary libraries

#Data Wrangling
import pandas as pd
from scipy import stats
import numpy as np
import os
import nltk
nltk.download('punkt')
from nltk import Text
from nltk.tokenize import word_tokenize, sent_tokenize, PunktSentenceTokenizer
from nltk.tokenize.punkt import PunktLanguageVars
from random import choices
import string
import matplotlib.pyplot as plt
from collections import Counter
import json
import re
import re
from collections import Counter
import time
import gensim #library needed for word2vec
import regex
import difflib
import itertools
import random 


#language detection
import langdetect
from langdetect import detect_langs, DetectorFactory, LangDetectException

#Multiprocessing
from joblib import Parallel, delayed

#for visualization
from scipy.spatial.distance import cosine
from sklearn.metrics import pairwise
from sklearn.manifold import MDS, TSNE

[nltk_data] Downloading package punkt to /Users/luki/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# Read JSON file into a DataFrame
data = pd.read_json('../../SGG_corpus_052024.json')
df_all = pd.DataFrame(data)
df = df_all

In [8]:
## 1 Nummern rauslöschen... 
def remove_numbers(s):
    return re.sub(r'\d+', '', s)
df.loc[:, 'text_cl'] = df['text'].apply(remove_numbers)

### 2 Klammern rauslöschen...
df['text_cl'] = df['text_cl'].str.replace(r'\(|\)', '', regex=True)
df['text_cl'] = df['text_cl'].str.replace('"', '')

In [9]:
df

,doc_id,text,name,year,inst,nr,contains_google,text_cl
0,SGG_01_1810,Verhandlungen\n der\nschwe...,SGG_01_1810.txt,1810,SGG,1,False,Verhandlungen\n der\nschwe...
1,SGG_02_1811,Verhandlungen\n der\nschwe...,SGG_02_1811.txt,1811,SGG,2,False,Verhandlungen\n der\nschwe...
2,SGG_03_1812,Verhandlungen\n Der\nschwei...,SGG_03_1812.txt,1812,SGG,3,False,Verhandlungen\n Der\nschwei...
3,SGG_04_1813,Verhandlungen\n der\nschwe...,SGG_04_1813.txt,1813,SGG,4,False,Verhandlungen\n der\nschwe...
4,SGG_05_1814,Verhandlungen\n der\nschweiz...,SGG_05_1814.txt,1814,SGG,5,False,Verhandlungen\n der\nschweiz...
...,...,...,...,...,...,...,...,...
84,ZfG_57_1918,Schweizerische Zeitschrif...,ZfG_57_1918.txt,1918,ZfG,57,False,Schweizerische Zeitschrif...
85,ZfG_58_1919,Schweizerische Zeitschrift\n ...,ZfG_58_1919.txt,1919,ZfG,58,False,Schweizerische Zeitschrift\n ...
86,ZfG_59_1920,Schweizerische Zeitschrift\n ...,ZfG_59_1920.txt,1920,ZfG,59,False,Schweizerische Zeitschrift\n ...
87,ZfG_60_1921,Schweizerische Zeitschrift\n ...,ZfG_60_1921.txt,1921,ZfG,60,False,Schweizerische Zeitschrift\n ...


In [14]:
def get_exclusive_punctuation_words(texts):
    end_punctuations = re.compile(r"[\.\!\?]['\"]?$")
    
    # Convert all words to lowercase once
    all_words_lower = ' '.join(texts).lower().split()
    all_words_set = set(all_words_lower)
    word_freq = Counter(all_words_lower)

    # Get words with end punctuation
    words_with_end_punctuation = [word for word in all_words_lower if end_punctuations.search(word)]
    
    # Strip punctuation from the end of these words
    words_stripped = [re.sub(end_punctuations, '', word) for word in words_with_end_punctuation]
    
    # Adjust here: check if the word frequency is greater than 5
    exclusive_punctuation_words = set(word for word, stripped_word in zip(words_with_end_punctuation, words_stripped) 
                                     if stripped_word not in all_words_set and word_freq[word] > 5)
    
    # Sort by frequency in descending order while ensuring each word appears only once
    sorted_exclusives = sorted(exclusive_punctuation_words, key=lambda x: word_freq[x], reverse=True)
    
    return sorted_exclusives

In [15]:
exclusives = get_exclusive_punctuation_words(df['text_cl'])
print("Exclusive words with punctuation:", exclusives)

Exclusive words with punctuation: ['.', '?', '!', 'a.-g.', 'pct.', 'eingetret.', 'j.-b.', 'p.t.', 'z.-k.', 'decbr.', 'ztr.', 'rpp.', 'p.-l.', ".'", 'sgr.', '„joh.', '\ufeffh.', 'l.st.', 'repräsentantenrathes.', 'p.m.', 'ehrw.', 'livr.-sterl.', 'a.g.', 'geford.', 'schul-inspector.', 'schulkomm.', 'assekuranzprämien.', 'j.-j.', 'a.-r.', 'hlr.', '*hr.', 'l.-st.', 'bzw.', 'r.-r.', 'wkt.', '□m.', 'geometr.', 'freiw.', 'haush.-k.', 'bretzwyl.', '»n.', 'j.-bb.', 'diac.', 'unehel.', ',.nro.', 'b.-k.', 'cv.', 'stuttg.', 'geschl.', 'doll.', 'b.-v.', 'm.d.', 'dr.dr.', 'thlrn.', 'iiz.', 'ohn.vorsch.', 'lunnern.', 'fräul.', '„dr.', 'unentgeltl.', 'm.g.', '\ufeffa.', 'b.-b.', 'manuftript.', '□f.', 'fr.-v.', 'a.dr.', 'hülfsver.', 'u.s.', 'h.h.', 'j.-l.', 'weisslingen.', 'ilz.', 'eidgenöss.', 'r.-k.', 'w.-v.', 'thl.', '„j.', 'sentantenrathes.', 'oktbr.', 'glätte-k.', 'landamm.', 's.-u.', 'präsentantenrathes.', 'eod.', 'mustersch.', 'a.-u.', 'haushsch.', 'anmerk.', 'bfr.', 'zimmermann-v.', 'haftg.', 's

In [16]:
exclusives = [element for element in exclusives if element not in ['.', '?', '!', ".'"]]
len(exclusives)

104

In [17]:
# Open file for writing
with open("exceptions_sentenizer.txt", "w", encoding="utf-8") as f:
    for item in exclusives:
        f.write("%s\n" % item)

In [20]:
## ERGÄNZUNG MIT ABKÜRZUNGSLISTE: https://github.com/jfilter/german-abbreviations/blob/master/german_abbreviations.txt
## SIEHE AUCH: https://github.com/adbar/German-NLP

with open("german_abbreviations.txt", 'r', encoding="utf-8") as file:
    german_abbrev = [line.strip() for line in file]

print(german_abbrev)

merged_list = list(set(exclusives + german_abbrev))
print(merged_list)

abbreviation_list = [abbreviation.rstrip('.') for abbreviation in merged_list]
abbreviation_list

['&c.', '*g*', '*gg*', '0.', '1 Chr', '1 Joh', '1 Kor', '1 Kön', '1 Makk', '1 Mos', '1 Petr', '1.', '10.', '2 Chr', '2 Joh', '2 Kor', '2 Kön', '2 Makk', '2 Mos', '2 Petr', '2.', '24/7', '2D', '3 Joh', '3 Makk', '3 Mos', '3-D', '3.', '3D', '4 Makk', '4 Mos', '4.', '4D', '5 Mos', '5.', '6.', '7.', '8.', '9.', '9/11', 'A', 'a', 'a c.', 'a d.', 'a v.', 'A.', 'a.', 'a. A.', 'a. a.', 'a. a. O.', 'A. B.', 'a. c. i.', 'a. Chr.', 'a. D.', 'a. d.', 'A. D.', 'a. d. D.', 'a. d. E.', 'A. d. Hrsg.', 'a. d. O.', 'a. f.', 'a. G.', 'a. i.', 'a. l. i. c.', 'a. M.', 'a. n. Chr.', 'a. n. g.', 'a. O.', 'a. o. Prof.', 'a. Rh.', 'a. St.', 'A. T.', 'a. t.', 'a. u. c.', 'a. Z.', 'a.D.', 'a.d.D.', 'AA', 'AAA', 'AAB', 'AABG', 'AAD', 'AAG', 'AAK', 'AAM', 'aaO', 'AAP', 'AAppO', 'AAR', 'AAS', 'AAV', 'AAÜG', 'AB', 'Abb.', 'ABC', 'abchas.', 'abds.', 'ABE', 'ABF', 'Abf.', 'AbfG', 'AbfKoBiV', 'Abfr.', 'ABG', 'Abg.', 'ABGB', 'abgek.', 'AbgG', 'abh.', 'Abh.', 'ABI', 'Abk.', 'ABl.', 'Abl.', 'ABl.EG', 'ABM', 'Abm.', 'ABMG'

['WVG',
 'CFO',
 'aengl',
 'marinespr',
 'E 553a',
 'BPjM',
 'MSR',
 'BayRS',
 'E 962',
 'BÖ',
 'KrimOI',
 'vergl',
 'Sbd',
 'Std',
 'ZOB',
 'DP',
 'etc. pp',
 'Yds',
 'LEGIDA',
 'Kw',
 'E 263',
 'EVP',
 'GKG',
 'a v',
 'BLG',
 'HR',
 '°F',
 'E 579',
 'alem',
 'VermA',
 'arcosh',
 'weisslingen',
 'E 406',
 'ETCS',
 'StKptLt',
 'E 943a',
 '1 Chr',
 'USt',
 '„dr',
 'RHG',
 'typogr',
 'BarwertVO',
 'EntschG',
 'Prof',
 'qm',
 'VwVfG',
 'Wa',
 '°R',
 'inkl',
 'Gefr',
 'altgr',
 'BER',
 'BAA',
 'idg',
 'nl',
 'AfD',
 '4 Makk',
 'bsd',
 'OGefr',
 'RAW',
 'SanS',
 'Adm',
 'SSD',
 'GRZ',
 'Hyp',
 'SprengG',
 'E 444',
 'eidgenöss',
 'Dres',
 'hdmdl',
 'AlhiRG',
 'k. u',
 'TQM',
 'BGB',
 'EEG',
 'Art.-Nr',
 'PersAuswG',
 'Be',
 'Apostr',
 'E 472d',
 'lfd. M',
 'md',
 'RBÜ',
 'ung',
 'DHH',
 'leg',
 'MdJ',
 'ungl',
 'FAVO',
 'indiff',
 'LGB',
 'tib',
 'WM',
 'arab',
 'E 200',
 'rhein',
 'RSHA',
 'YUGStrGHG',
 'EIZ',
 'französ',
 'p. a',
 'wld',
 'BMHS',
 'ged',
 'EP',
 'LÜ',
 'QZ',
 'mniederd',
 

In [23]:
sentence_tokenizer = nltk.data.load("tokenizers/punkt/german.pickle")
sentence_tokenizer._params.abbrev_types.update(abbreviation_list)

## Neue Funktion, welche Ausnahmen berücksichtigt. 
def tokenize_text(text):
    # Update the sentence tokenizer with the modified abbreviation list
    sentence_tokenizer._params.abbrev_types.update(abbreviation_list)
    
    # Tokenize the text using the modified sentence tokenizer
    split_sentences = sentence_tokenizer.tokenize(text)
    
    return split_sentences

In [24]:
df['sentences'] = df['text_cl'].apply(tokenize_text)

In [29]:
df['sentences'][1][100:105]

['Nicht schauender, nicht Genießender nur,\n       Mitfreuender solltest du werden,\n       Mit würdigen sollst du was ist und geschieht,\n       Mitschöpfen und herrschen auf Erden.',
 'Untrüglich leuchtet himmlisches Licht\n       In deine geheiligte Seele,\n       Damit beym Wagen des Dingewerths\n       Dich leitende Klarheit nicht fehle.',
 'Dein Blick hangt freudig am Ebenmaß,\n       Du fühlst, ob Formen gefallen,\n       Dirifchmeichelt der Färbungen Lieblichkeit,\n       Der Lieder harmonisches Wallen.',
 "In's Innere der Wesen dringet dein Blick, '\n       Dich fesselt Schönheit der Seele,\n       Dich spricht der Geist aus Handlungen an,\n       Du kennst der Liebe Befehle.",
 'Du zählest die Wesen, und reihest sie,\n       Du missest, was Augen nicht schauen,\n       Du rufst aus der Gruft die Vergangenheit her.']

In [31]:
# Function to tokenize sentences into words
def tokenize_sentences_into_words(sentence_list):
    tokenized_sentences = [word_tokenize(sentence, language='german') for sentence in sentence_list]
    return tokenized_sentences

In [33]:
# Apply the function to the 'sentences' column and save results in 'tokenized_sentences'
df['tokenized_sentences'] = df['sentences'].apply(tokenize_sentences_into_words)
df['tokenized_sentences'][1][100:105]

[['Nicht',
  'schauender',
  ',',
  'nicht',
  'Genießender',
  'nur',
  ',',
  'Mitfreuender',
  'solltest',
  'du',
  'werden',
  ',',
  'Mit',
  'würdigen',
  'sollst',
  'du',
  'was',
  'ist',
  'und',
  'geschieht',
  ',',
  'Mitschöpfen',
  'und',
  'herrschen',
  'auf',
  'Erden',
  '.'],
 ['Untrüglich',
  'leuchtet',
  'himmlisches',
  'Licht',
  'In',
  'deine',
  'geheiligte',
  'Seele',
  ',',
  'Damit',
  'beym',
  'Wagen',
  'des',
  'Dingewerths',
  'Dich',
  'leitende',
  'Klarheit',
  'nicht',
  'fehle',
  '.'],
 ['Dein',
  'Blick',
  'hangt',
  'freudig',
  'am',
  'Ebenmaß',
  ',',
  'Du',
  'fühlst',
  ',',
  'ob',
  'Formen',
  'gefallen',
  ',',
  'Dirifchmeichelt',
  'der',
  'Färbungen',
  'Lieblichkeit',
  ',',
  'Der',
  'Lieder',
  'harmonisches',
  'Wallen',
  '.'],
 ['In',
  "'s",
  'Innere',
  'der',
  'Wesen',
  'dringet',
  'dein',
  'Blick',
  ',',
  "'",
  'Dich',
  'fesselt',
  'Schönheit',
  'der',
  'Seele',
  ',',
  'Dich',
  'spricht',
  'der',
  

In [34]:
def remove_punctuation(tokens):
    # Pattern to match any punctuation character in any language
    # \p{P} matches any kind of punctuation character
    # \p{M} might be necessary to keep combining characters like accents => EVENTUELL einbauen...
    pattern = regex.compile(r"\p{P}")

    # For each token, remove punctuation characters
    # and keep tokens that still contain some content after cleanup
    return [pattern.sub('', token) for token in tokens if pattern.sub('', token)]

def process_nested_list(nested_list):
    return [remove_punctuation(sentence) for sentence in nested_list]

# Apply the function on each nested list in 'tokenized_sentences'
df['cleaned_sentences'] = df['tokenized_sentences'].apply(process_nested_list)
df['cleaned_sentences'][1][100:103]

[['Nicht',
  'schauender',
  'nicht',
  'Genießender',
  'nur',
  'Mitfreuender',
  'solltest',
  'du',
  'werden',
  'Mit',
  'würdigen',
  'sollst',
  'du',
  'was',
  'ist',
  'und',
  'geschieht',
  'Mitschöpfen',
  'und',
  'herrschen',
  'auf',
  'Erden'],
 ['Untrüglich',
  'leuchtet',
  'himmlisches',
  'Licht',
  'In',
  'deine',
  'geheiligte',
  'Seele',
  'Damit',
  'beym',
  'Wagen',
  'des',
  'Dingewerths',
  'Dich',
  'leitende',
  'Klarheit',
  'nicht',
  'fehle'],
 ['Dein',
  'Blick',
  'hangt',
  'freudig',
  'am',
  'Ebenmaß',
  'Du',
  'fühlst',
  'ob',
  'Formen',
  'gefallen',
  'Dirifchmeichelt',
  'der',
  'Färbungen',
  'Lieblichkeit',
  'Der',
  'Lieder',
  'harmonisches',
  'Wallen'],
 ['In',
  's',
  'Innere',
  'der',
  'Wesen',
  'dringet',
  'dein',
  'Blick',
  'Dich',
  'fesselt',
  'Schönheit',
  'der',
  'Seele',
  'Dich',
  'spricht',
  'der',
  'Geist',
  'aus',
  'Handlungen',
  'an',
  'Du',
  'kennst',
  'der',
  'Liebe',
  'Befehle'],
 ['Du',
  

In [35]:
#### 4 Anhand von Wortliste die OCR-Qualität berechnen: 

### Die tokenisierten Sätze zu Lowercase machen. 
# Function to convert tokens in a sentence to lowercase
def lowercase_tokens(sentence):
    return [token.lower() for token in sentence]

# Function to apply the lowercase conversion on nested lists
def process_nested_list_for_lower(nested_list):
    return [lowercase_tokens(sentence) for sentence in nested_list]

# Apply the function on each nested list in 'cleaned_sentences'
df['cleaned_sentences_LC'] = df['cleaned_sentences'].apply(process_nested_list_for_lower)

### IMPORT DICTIONARY:
german_word_list = []

with open('merged_word_list_incl_French.txt', 'r', encoding='utf-8') as fp:
    for line in fp:
        x = line[:-1]

        # add current item to the list
        german_word_list.append(x)

# display list
german_word_list_LC = [word.lower() for word in german_word_list]
print(len(german_word_list))

4773539


In [45]:
#### Abgleichen, wie viel Prozent der Token pro Text in der Wortliste vorkommen

from itertools import chain

# Convert german_word_list_LC to a set for faster look-up
german_word_set_LC = set(german_word_list_LC)

def calculate_percentage(tokens, german_word_set_LC):
    # Calculate the percentage of tokens in the word list
    matching_tokens = [token for token in tokens if token in german_word_set_LC]
    percentage = (len(matching_tokens) / len(tokens)) * 100 if tokens else 0
    return percentage

# Flatten the nested lists and calculate the percentage
df['ocr_quality'] = df['cleaned_sentences_LC'].apply(lambda nested_list: calculate_percentage(list(chain.from_iterable(nested_list)), german_word_set_LC))

# Check and print if any entries have OCR quality below 90
if (df['ocr_quality'] < 90).any():
    print(df[df['ocr_quality'] < 90])
else:
    print("No entries with OCR quality below 90.")

No entries with OCR quality below 90.


In [46]:
#Berechne die Gesamtzahl der Tokens
total_tokens = df['cleaned_sentences'].apply(lambda sentences: sum(len(sentence) for sentence in sentences)).sum()
print(f"Es bleiben {total_tokens} Tokens in der 'tokenized_sentences' Spalte übrig.")

Es bleiben 11536759 Tokens in der 'tokenized_sentences' Spalte übrig.


In [59]:
### FRANZÖSISCHE SÄTZE RAUSFILTERN
##### Zunächst nur mit zufällig ausgewählten Sätzen

sgg_df = df
all_sentences = list(itertools.chain(*sgg_df['cleaned_sentences']))
random_sample = random.sample(all_sentences, min(1000, len(all_sentences)))

excluded_sentences = []
updated_sentences = []

for sentence_list in sgg_df['cleaned_sentences']:
    updated_sentence_list = []
    for sentence in sentence_list:
        if sentence in random_sample:
            sentence_text = ' '.join(sentence)
            # Check if the sentence has enough content (e.g., not just a few words)
            if len(sentence_text.split()) > 3:
                try:
                    # Detect language and probability
                    probabilities = detect_langs(sentence_text)
                    # Check if French is detected with probability > 95%
                    if any(lang.lang == 'fr' and lang.prob > 0.95 for lang in probabilities):
                        excluded_sentences.append(sentence_text)
                        continue
                except LangDetectException:
                    pass  # Handle the exception or log if needed
        updated_sentence_list.append(sentence)
    updated_sentences.append(updated_sentence_list)

# Update the DataFrame
sgg_df['cleaned_sentences'] = updated_sentences

# Save the excluded sentences to a text file
with open('excluded_sentences.txt', 'w', encoding='utf-8') as file:
    file.write('#'.join(excluded_sentences))

print("Excluded sentences saved to excluded_sentences.txt")


Excluded sentences saved to excluded_sentences.txt


In [62]:
#### Über den gesamten Datensatz

excluded_sentences = []
updated_sentences = []

for sentence_list in sgg_df['cleaned_sentences']:
    updated_sentence_list = []
    for sentence in sentence_list:
        sentence_text = ' '.join(sentence)
        # Check if the sentence has enough content (e.g., not just a few words)
        if len(sentence_text.split()) > 3:
            try:
                # Detect language and probability
                probabilities = detect_langs(sentence_text)
                # Check if French is detected with probability > 95%
                if any(lang.lang == 'fr' and lang.prob > 0.95 for lang in probabilities):
                    excluded_sentences.append(sentence_text)
                    continue
            except LangDetectException:
                pass  # Handle the exception or log if needed
        updated_sentence_list.append(sentence)
    updated_sentences.append(updated_sentence_list)

# Update the DataFrame
sgg_df['cleaned_sentences'] = updated_sentences

# Save the excluded sentences to a text file
with open('excluded_french_sentences.txt', 'w', encoding='utf-8') as file:
    file.write('#'.join(excluded_sentences))

print("Excluded sentences saved to excluded_french_sentences.txt")

Excluded sentences saved to excluded_french_sentences.txt


In [63]:
len(excluded_sentences)

37672

In [64]:
total_sentences = sum(len(sentence_list) for sentence_list in sgg_df['cleaned_sentences'])
print("Total number of sentences:", total_sentences)

Total number of sentences: 619963


In [65]:
### Zwischenschritt: Datensatz speichern
sgg_df.to_json("../data/corpus_clened_preprocessed_NOFRENCH.json")
# Read JSON file into a DataFrame
data = pd.read_json('../data/corpus_clened_preprocessed_NOFRENCH.json')
df = pd.DataFrame(data)
df

,doc_id,text,name,year,inst,nr,contains_google,text_cl,sentences,tokenized_sentences,cleaned_sentences,cleaned_sentences_LC,ocr_quality
0,SGG_01_1810,Verhandlungen\n der\nschwe...,SGG_01_1810.txt,1810,SGG,1,False,Verhandlungen\n der\nschwe...,[Verhandlungen\n der\nschw...,"[[Verhandlungen, der, schweizerischen, gemeinn...","[[Verhandlungen, der, schweizerischen, gemeinn...","[[verhandlungen, der, schweizerischen, gemeinn...",97.397442
1,SGG_02_1811,Verhandlungen\n der\nschwe...,SGG_02_1811.txt,1811,SGG,2,False,Verhandlungen\n der\nschwe...,[Verhandlungen\n der\nschw...,"[[Verhandlungen, der, schweizerisch, gemeinnüt...","[[Verhandlungen, der, schweizerisch, gemeinnüt...","[[verhandlungen, der, schweizerisch, gemeinnüt...",96.186141
2,SGG_03_1812,Verhandlungen\n Der\nschwei...,SGG_03_1812.txt,1812,SGG,3,False,Verhandlungen\n Der\nschwei...,[Verhandlungen\n Der\nschwe...,"[[Verhandlungen, Der, schweizerisch, gemeinnüß...","[[Verhandlungen, Der, schweizerisch, gemeinnüß...","[[verhandlungen, der, schweizerisch, gemeinnüß...",94.035670
3,SGG_04_1813,Verhandlungen\n der\nschwe...,SGG_04_1813.txt,1813,SGG,4,False,Verhandlungen\n der\nschwe...,[Verhandlungen\n der\nschw...,"[[Verhandlungen, der, schweizerisch, gemeinnüt...","[[Verhandlungen, der, schweizerisch, gemeinnüt...","[[verhandlungen, der, schweizerisch, gemeinnüt...",95.572470
4,SGG_05_1814,Verhandlungen\n der\nschweiz...,SGG_05_1814.txt,1814,SGG,5,False,Verhandlungen\n der\nschweiz...,[Verhandlungen\n der\nschwei...,"[[Verhandlungen, der, schweizerischgemeinnützi...","[[Verhandlungen, der, schweizerischgemeinnützi...","[[verhandlungen, der, schweizerischgemeinnützi...",96.472455
...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,ZfG_57_1918,Schweizerische Zeitschrif...,ZfG_57_1918.txt,1918,ZfG,57,False,Schweizerische Zeitschrif...,[Schweizerische Zeitschri...,"[[Schweizerische, Zeitschrift, für, Gemeinnütz...","[[Schweizerische, Zeitschrift, für, Gemeinnütz...","[[schweizerische, zeitschrift, für, gemeinnütz...",96.062765
85,ZfG_58_1919,Schweizerische Zeitschrift\n ...,ZfG_58_1919.txt,1919,ZfG,58,False,Schweizerische Zeitschrift\n ...,[Schweizerische Zeitschrift\n ...,"[[Schweizerische, Zeitschrift, für, Gemeinnütz...","[[Schweizerische, Zeitschrift, für, Gemeinnütz...","[[schweizerische, zeitschrift, für, gemeinnütz...",96.093633
86,ZfG_59_1920,Schweizerische Zeitschrift\n ...,ZfG_59_1920.txt,1920,ZfG,59,False,Schweizerische Zeitschrift\n ...,[Schweizerische Zeitschrift\n ...,"[[Schweizerische, Zeitschrift, für, Gemeinnütz...","[[Schweizerische, Zeitschrift, für, Gemeinnütz...","[[schweizerische, zeitschrift, für, gemeinnütz...",96.077273
87,ZfG_60_1921,Schweizerische Zeitschrift\n ...,ZfG_60_1921.txt,1921,ZfG,60,False,Schweizerische Zeitschrift\n ...,[Schweizerische Zeitschrift\n ...,"[[Schweizerische, Zeitschrift, für, Gemeinnütz...","[[Schweizerische, Zeitschrift, für, Gemeinnütz...","[[schweizerische, zeitschrift, für, gemeinnütz...",96.158120


In [66]:
total_tokens_count = sum(len(sentence) for sentence_list in df['cleaned_sentences'] for sentence in sentence_list)
print(total_tokens_count)

10562429


In [67]:
### Ältere Schreibweisen korrigieren (Armuth mit Armut ersetzen) 
def replace_tokens(sentence, replacements, replacement_counts):
    new_sentence = []
    for token in sentence:
        if token in replacements:
            new_sentence.append(replacements[token])
            replacement_counts[token] += 1
        else:
            new_sentence.append(token)
    return new_sentence

# Dictionary mapping old tokens to new tokens
replacements = {
    'Armuth': 'Armut',
    'Reichthum': 'Reichtum',
    'Concurrenz': 'Konkurrenz'
    # Add more replacements as needed, e.g., 'OldWord': 'NewWord',
}

# Initialize a dictionary to count replacements for each token
replacement_counts = {token: 0 for token in replacements}

# Iterate over each text
for text in df['cleaned_sentences']:
    # Iterate over each sentence in the text
    for i, sentence in enumerate(text):
        text[i] = replace_tokens(sentence, replacements, replacement_counts)

# Print the replacement counts for each token
for token, count in replacement_counts.items():
    print(f"'{token}' was replaced {count} times")

'Armuth' was replaced 999 times
'Reichthum' was replaced 266 times
'Concurrenz' was replaced 132 times


In [68]:
#CHECK FREQUENCIES OF TARGET WORD
new_words = list(replacements.values())

# Flatten all tokens from all sentences into a single list
all_tokens = []
for text in df['cleaned_sentences']:
    for sentence in text:
        all_tokens.extend(sentence)

# Use Counter to count occurrences of each token
token_counts = Counter(all_tokens)

# Filter the counts to include only the new words
new_word_counts = {word: token_counts[word] for word in new_words}

# Print the count for each new word
for word, count in new_word_counts.items():
    print(f"Count of '{word}': {count}")

Count of 'Armut': 1348
Count of 'Reichtum': 326
Count of 'Konkurrenz': 688


In [71]:
df.to_json("../data/corpus_final.json")